# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [5]:
# Features and target
X = fires_dt.drop(columns=['area'])
y = fires_dt['area']

# Identify categorical and numerical columns
categorical_columns = ['month', 'day']
numerical_columns = [col for col in X.columns if col not in categorical_columns]
#numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns.drop(categorical_columns)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [6]:
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ]
)


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [7]:
preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', MinMaxScaler()),
            ('transform', PowerTransformer())
        ]), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ]
)


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [8]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline


In [9]:
# Pipeline A = preproc1 + baseline
pipe_a = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', KNeighborsRegressor())
])

In [10]:
# Pipeline B = preproc2 + baseline
pipe_b = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', KNeighborsRegressor())
])


In [11]:
# Pipeline C = preproc1 + advanced model
pipe_c = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [13]:
# Pipeline D = preproc2 + advanced model
pipe_d = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=42))
])   

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [14]:
# Parameter grids for grid search
param_grid_a = {
    'regressor__n_neighbors': [3, 5, 7, 9]
}

param_grid_b = param_grid_a  # Same for baseline with preproc2

param_grid_c = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20]
}

param_grid_d = param_grid_c  # Same for advanced regressor with preproc2


In [15]:
# Grid search for each pipeline
grid_a = GridSearchCV(pipe_a, param_grid_a, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_b = GridSearchCV(pipe_b, param_grid_b, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_c = GridSearchCV(pipe_c, param_grid_c, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_d = GridSearchCV(pipe_d, param_grid_d, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Fit the pipelines
print("Tuning Pipeline A (preproc1 + baseline)...")
grid_a.fit(X_train, y_train)

print("Tuning Pipeline B (preproc2 + baseline)...")
grid_b.fit(X_train, y_train)

print("Tuning Pipeline C (preproc1 + advanced)...")
grid_c.fit(X_train, y_train)

print("Tuning Pipeline D (preproc2 + advanced)...")
grid_d.fit(X_train, y_train)


Tuning Pipeline A (preproc1 + baseline)...


c:\Users\prsrivastava\.conda\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(
c:\Users\prsrivastava\.conda\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


Tuning Pipeline B (preproc2 + baseline)...
Tuning Pipeline C (preproc1 + advanced)...


c:\Users\prsrivastava\.conda\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Tuning Pipeline D (preproc2 + advanced)...


c:\Users\prsrivastava\.conda\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          MinMaxScaler()),
                                                                                         ('transform',
                                                                                          PowerTransformer())]),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [None, 10, 20],
                         'regressor__n_estimators': [50, 100, 200]},
             scoring='neg_root_mean_squared_error')

In [16]:
# Helper function to extract results
def summarize_results(grid, name):
    print(f"\n{name}")
    print(f"Best Parameters: {grid.best_params_}")
    print(f"Best CV RMSE: {-grid.best_score_:.4f}")
    return grid.best_estimator_

# Summarize results for each pipeline
best_a = summarize_results(grid_a, "Pipeline A")
best_b = summarize_results(grid_b, "Pipeline B")
best_c = summarize_results(grid_c, "Pipeline C")
best_d = summarize_results(grid_d, "Pipeline D")



Pipeline A
Best Parameters: {'regressor__n_neighbors': 3}
Best CV RMSE: nan

Pipeline B
Best Parameters: {'regressor__n_neighbors': 3}
Best CV RMSE: nan

Pipeline C
Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}
Best CV RMSE: nan

Pipeline D
Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}
Best CV RMSE: nan


In [17]:
# Test set performance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model_on_test(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test MAE: {mae:.4f}")
    print(f"Test R^2: {r2:.4f}")

print("\nTest Performance of Best Models:")
print("Pipeline A:")
evaluate_model_on_test(best_a, X_test, y_test)

print("\nPipeline B:")
evaluate_model_on_test(best_b, X_test, y_test)

print("\nPipeline C:")
evaluate_model_on_test(best_c, X_test, y_test)

print("\nPipeline D:")
evaluate_model_on_test(best_d, X_test, y_test)



Test Performance of Best Models:
Pipeline A:
Test RMSE: 110.2619
Test MAE: 24.4611
Test R^2: -0.0314

Pipeline B:
Test RMSE: 110.1176
Test MAE: 24.1434
Test R^2: -0.0287

Pipeline C:
Test RMSE: 109.4693
Test MAE: 26.6703
Test R^2: -0.0166

Pipeline D:
Test RMSE: 109.7183
Test MAE: 26.5887
Test R^2: -0.0212


In [18]:
# Helper function to summarize grid search results
def summarize_grid_search_results(grid, name):
    print(f"\n{name}")
    print(f"Best Parameters: {grid.best_params_}")
    print(f"Best Cross-Validated RMSE: {-grid.best_score_:.4f}")
    return grid.best_estimator_

# Summarize each pipeline
best_a = summarize_grid_search_results(grid_a, "Pipeline A (Preproc1 + Baseline)")
best_b = summarize_grid_search_results(grid_b, "Pipeline B (Preproc2 + Baseline)")
best_c = summarize_grid_search_results(grid_c, "Pipeline C (Preproc1 + Advanced)")
best_d = summarize_grid_search_results(grid_d, "Pipeline D (Preproc2 + Advanced)")



Pipeline A (Preproc1 + Baseline)
Best Parameters: {'regressor__n_neighbors': 3}
Best Cross-Validated RMSE: nan

Pipeline B (Preproc2 + Baseline)
Best Parameters: {'regressor__n_neighbors': 3}
Best Cross-Validated RMSE: nan

Pipeline C (Preproc1 + Advanced)
Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}
Best Cross-Validated RMSE: nan

Pipeline D (Preproc2 + Advanced)
Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 50}
Best Cross-Validated RMSE: nan


# Evaluate

+ Which model has the best performance?

After executing the GridSearch and evaluating the four pipelines, compare their Test RMSE scores. The model with the lowest RMSE on the test set is the best-performing model.

In this case, Pipeline C (Preproc1 + RandomForestRegressor) has the best performance.

# Export

+ Save the best performing model to a pickle file.

In [20]:
import pickle

# Save the best model
best_model = best_c  # Replace with the actual best model pipeline
pickle_file = "C:/Users/prsrivastava/DSI_Demo/production/05_src/data/fires/best_model.pkl"

with open(pickle_file, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Best model saved to {pickle_file}.")


Best model saved to C:/Users/prsrivastava/DSI_Demo/production/05_src/data/fires/best_model.pkl.


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [24]:
import shap

*(Answer here.)*

Best Model: Use Test RMSE to identify the best pipeline.
Export: Save the best-performing model as a .pkl file.
Explain Using SHAP:

    For a specific observation: Use SHAP values to identify key predictors for that prediction.
    Globally: Use a SHAP summary plot to rank feature importance.
    Feature removal: Use SHAP insights to identify and test the effect of removing less important features.

In [28]:
# Transform the data using the pipeline's preprocessing step
X_test_transformed = best_model.named_steps['preprocessing'].transform(X_test)


In [31]:
# Select a single test observation (preprocessed)
test_observation_transformed = X_test_transformed[0]  # Assuming X_test_transformed is a NumPy array

# SHAP Explainer
explainer = shap.TreeExplainer(best_model.named_steps['regressor'])
shap_values = explainer.shap_values(X_test_transformed)

# Visualize SHAP values for the observation
#shap.force_plot(
    #explainer.expected_value,
    #shap_values[0],  # Corresponding SHAP values
    #test_observation_transformed
#)


## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.